# Find centers

In [ ]:
import pandas as pd
from geopandas import GeoDataFrame, points_from_xy
from numpy import diff
from pandas import DataFrame, NamedAgg
from shapely import LinearRing

from boilercv.data.sets import get_contours_df, get_dataset
from boilercv.docs import init
from boilercv.models.params import PARAMS
from boilercv.stages.experiments.e230920_subcool import EXP, GBC, THERMAL_DATA

init()

In [ ]:
TIME = "2023-09-20T17:14:18"

In [ ]:
PATH_TIME = TIME.replace(":", "-")
PATH = PARAMS.paths.experiments / f"{EXP}/{PATH_TIME}.h5"

video = get_dataset(PATH_TIME, stage="filled")["video"]

# Conversion factors
PX_PER_M = 20997.3753  # (px/m)
PX_PER_MM = PX_PER_M / 1000  # (px/mm)

# Informed by the actual water temperature data
DATA = pd.read_hdf(THERMAL_DATA)
# TODO: Timezone
SUBCOOLING = DATA.subcool[TIME]

FRAMETIME = diff(video.time.values).mean()

print(
    f"Rate = {1e3*FRAMETIME:.2f} ms",
    f"Scale = {PX_PER_MM:.2f} px/mm",
    f"Subcool = {SUBCOOLING:.2f} K",
    sep="\n",
)

In [ ]:
source_ds = get_dataset(PATH_TIME, stage="filled")
contours = (
    DataFrame(columns=["xpx", "ypx"])
    .assign(**get_contours_df(PATH_TIME))
    .rename(axis="columns", mapper=dict(xpx="x", ypx="y"))
    .reset_index()
    .assign(
        count=lambda df: df.groupby(["frame", "contour"], **GBC).x.transform("count")
    )
    .query("count > 3")
    .drop(columns="count")
    .assign(geometry=lambda df: points_from_xy(df.x, df.y))
    .groupby(["frame", "contour"], **GBC)  # type: ignore  # pyright 1.1.333
    .agg(
        x=NamedAgg(column="x", aggfunc="mean"),
        y=NamedAgg(column="x", aggfunc="mean"),
        centroid=NamedAgg(
            column="geometry", aggfunc=lambda df: LinearRing(df).centroid
        ),
    )
)
contours

In [ ]:
centers = (
    GeoDataFrame(contours)
    # .assign(x=lambda df: df.centroid.x, y=lambda df: df.centroid.y)
    .loc[:, ["frame", "contour", "x", "y"]]
)
centers